In [12]:
# Auto-reload internal modules
%load_ext autoreload
%autoreload 2

# External modules
import numpy as np

# Internal modules
from cosmotd.domain_walls import run_domain_wall_simulation
from cosmotd.cosmic_strings import run_cosmic_string_simulation
from cosmotd.charged_domain_walls import run_charged_domain_walls_simulation
from cosmotd.junctions import run_junction_simulation
from cosmotd.plot import MplMp4Plotter, MplPngPlotter, MplNotebookPlotter, MockPlotter
from cosmotd.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Size of simulation grid
N = 50
# Model parameters
beta = 1
eta_phi = 1
eta_sigma = np.sqrt(3) / 2
lam_phi = 2
lam_sigma = 2
charge_density = 0.1
# RNG seed
seed = 23213241
# Run time
run_time = None

In [16]:
# Domain walls simulation
run_domain_wall_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, MplMp4Plotter, seed, run_time)

100%|██████████| 250/250 [01:59<00:00,  2.09it/s]


In [19]:
# Cosmic strings simulation
run_cosmic_string_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, MplMp4Plotter, seed, run_time)

100%|██████████| 250/250 [01:14<00:00,  3.33it/s]


In [20]:
# Charged domain walls simulation
run_charged_domain_walls_simulation(
    N, DX, DT, ALPHA_2D, beta, eta_phi, eta_sigma, lam_phi, lam_sigma, charge_density, RADIATION_ERA, MplMp4Plotter, seed, run_time
)

100%|██████████| 250/250 [01:46<00:00,  2.35it/s]


In [21]:
# Pentavac model simulation with X-type and Y-type junctions
run_junction_simulation(N, 0.5, 0.1, ALPHA_2D, 0.1, RADIATION_ERA, MplMp4Plotter, seed, run_time)

100%|██████████| 125/125 [01:05<00:00,  1.90it/s]
